# STL file display

This notebook shows how to display STL files used for 3D printing in a notebook.

Displaying relies on the numpy-stl and the ipyvolume libraries.

Upload an STL **ascii** file (*binary files not supported*)

In [1]:
# imports
from stl import mesh
from traitlets import dlink
import ipyvolume as ipv
import ipywidgets as widgets
from IPython.display import display
from ipyupload import FileUpload
from io import StringIO

# widgets
file_picker = FileUpload(accept='*.stl')
file_name = widgets.Text()
wireframe_flag = widgets.Checkbox(
            value=False,
            description='Wireframe',
            disabled=False)
color_picker = widgets.ColorPicker(
            concise=False,
            description='Pick a color',
            value='white',
            disabled=False)
vbox = widgets.VBox([widgets.HBox([file_picker, file_name]), color_picker, wireframe_flag])

# display STL file
def show_stl(file_content, wireframe, color):
    # bridge: FileUpload - stl.mesh
    content = next(iter(file_content.values()))['content']
    io = StringIO()
    io.write(content.decode('utf-8'))
    io.seek(0)
    name, data = mesh.Mesh.load(io)
    m = mesh.Mesh(data, True, name=name, speedups=True)
    
    ipv.figure()
    ipv.style.use('minimal')
    m2 = ipv.plot_mesh(m.x, m.z, m.y, color=color, wireframe=wireframe)
    return ipv.gcc()

# get filename from FileUpload dict
def get_name(file_upload):
    if len(file_upload) == 0:
        return ''
    return next(iter(file_upload.values()))['metadata']['name']

def change_input(change):
    global vbox, file_picker, file_name, wireframe_flag, color_picker
    mymesh = show_stl(file_picker.value, wireframe_flag.value, color_picker.value)
    vbox.children = (widgets.HBox([file_picker, file_name]), wireframe_flag, color_picker, mymesh)

# links between widgets
color_picker.observe(change_input, 'value')
dlink((file_picker, 'value'), (file_name, 'value'), get_name)
wireframe_flag.observe(change_input, 'value')
file_picker.observe(change_input, 'value')

vbox